In [1]:
#inputs
input_file = 'globalTestFormat.csv' #the name of the input file

mrt_title = 'ACCEPT 2.0: A compilation of X-ray gas properties for clusters of galaxies observed by the Chandra X-ray Observatory' #the title of the paper
mrt_authors = 'Donahue M., Baldi A., Frisbie R., Koeppe D., Voit G.M.' #the authors of the paper
mrt_table = 'Global Properties' #name of the table

In [2]:
from astropy.table import Table
import pandas as pd
import numpy as np
import tempfile

#important bit about checked units: https://journals.aas.org/mrt-labels/#checked

df = pd.read_csv(input_file)
unitmap = df.to_dict(orient='records')[0] #create a map for the units of each column
descriptionmap = df.to_dict(orient='records')[1] #create a map for the description of each column
df = df.drop([0]) #drops the units row
df = df.drop([1]) #drops the description row

tempcsv = tempfile.NamedTemporaryFile() #writes a temporary csv file to be read by astropy, this fixes an issue where directly feeding astropy a pandas df doesn't correctly assign datatypes
df.to_csv(tempcsv,index=False)

csv = Table.read(tempcsv,format='csv',units=unitmap,descriptions=descriptionmap)

csv.write(input_file[:-4]+'.dat', overwrite=True, format='mrt')
with open(input_file[:-4]+'.dat', 'r+') as file:
    read_content=file.read()
    read_content=read_content.replace('Title:', 'Title: ' + mrt_title)
    read_content=read_content.replace('Authors:', 'Authors: '+ mrt_authors)
    read_content=read_content.replace('Table:', 'Table: '+ mrt_table)
    read_content=read_content.replace('table.dat', input_file[:-4]+'.dat')
    file.seek(0)
    file.write(read_content)

pd.read_csv(input_file,header=[0,1],skiprows=[2]).to_html(input_file[:-4]+'.html',index=False)